![](../MISC/unnamed_1.png)

# Analysing Customer Churn Data - Predicting Behavior to Retain Customers



Rayyan Minhaj (Data Science Intern - 10Pearls)

---

## Module 3: Database Schema, SQL, and Queries
Using SQL to create tables, perform data ingestion, call queries, and other various action items.

In [2]:
import psycopg2

In [3]:
conn = psycopg2.connect(host="localhost", dbname="10Pearls-Customer-Churn-Data", user="postgres", password="1234", port=5432)
cur = conn.cursor()

![](notebook_imgs/ERD.png)

## 1. Creating Schema & Tables 

In [24]:
cur.execute("""
    CREATE TABLE Customers (
    customer_id VARCHAR(50) PRIMARY KEY,
    gender VARCHAR(10),
    senior_citizen BOOLEAN,
    partner BOOLEAN,
    dependents BOOLEAN
);

    CREATE TABLE Services (
    customer_id VARCHAR(50),
    phone_service BOOLEAN,
    multiple_lines BOOLEAN,
    internet_service VARCHAR(50),
    online_security BOOLEAN,
    online_backup BOOLEAN,
    device_protection BOOLEAN,
    tech_support BOOLEAN,
    streaming_tv BOOLEAN,
    streaming_movies BOOLEAN,
            
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

CREATE TABLE Billing (
    customer_id VARCHAR(50),
    contract VARCHAR(50),
    paperless_billing BOOLEAN,
    payment_method VARCHAR(50),
    monthly_charges DECIMAL,
    total_charges DECIMAL,
    tenure INT,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);
            
    CREATE TABLE Churn_Predictions (
    customer_id VARCHAR(50),
    churn_prediction BOOLEAN,
    
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);        


""")

In [25]:
conn.commit()

![table_png.png](notebook_imgs/table_png.png)

### Services Table
![table_png.png](notebook_imgs/services_tbl.png)
### Customers Table
![table_png.png](notebook_imgs/customers_tbl.png)
### Churn Predictions Table
![table_png.png](notebook_imgs/churn_tbl.png)
### Billings Table
![table_png.png](notebook_imgs/billing_tbl.png)

## 2. Data Ingestion

In [27]:
import pandas as pd

raw_data = pd.read_csv('Raw_Data.csv')
telco_customer_churn = pd.read_csv('TelcoCustomerChurn.csv')
prediction_results = pd.read_csv('Prediction_Results.csv')

In [28]:
def insert_data(table_name, dataframe):
    for i, row in dataframe.iterrows():
        placeholders = ', '.join(['%s'] * len(row))
        columns = ', '.join(dataframe.columns)
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cur.execute(sql, tuple(row))
    conn.commit()

### (i) Inserting into Customers Table

In [32]:
customers_data = telco_customer_churn[['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents']]
customers_data.columns = ['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents']
customers_data['senior_citizen'] = customers_data['senior_citizen'].astype(bool)  #Convert to boolean
customers_data['partner'] = customers_data['partner'].map({'Yes': True, 'No': False})  #Convert to boolean
customers_data['dependents'] = customers_data['dependents'].map({'Yes': True, 'No': False})  #Convert to boolean
insert_data('customers', customers_data)

C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\3527777041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_data['senior_citizen'] = customers_data['senior_citizen'].astype(bool)  #Convert to boolean
C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\3527777041.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customers_data['partner'] = customers_data['partner'].map({'Yes': True, 'No': False})  #Convert to boolean
C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\3527777041.py:5: SettingWithCopyW

![table_png.png](notebook_imgs/customers_filled.png)

### (ii) Inserting into Services Table

In [33]:
services_data = telco_customer_churn[['customerID', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']]
services_data.columns = ['customer_id', 'phone_service', 'multiple_lines', 'internet_service', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies']
services_data = services_data.applymap(lambda x: True if x == 'Yes' else False if x == 'No' else x)  #Convert 'Yes'/'No' to boolean
insert_data('services', services_data)

C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\1937519431.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  services_data = services_data.applymap(lambda x: True if x == 'Yes' else False if x == 'No' else x)  # Convert 'Yes'/'No' to boolean


![table_png.png](notebook_imgs/services_filled.png)

### (iii) Inserting into Billings Table

In [48]:
billing_data = telco_customer_churn[['customerID', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'tenure']]
billing_data.columns = ['customer_id', 'contract', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'tenure']
billing_data['paperless_billing'] = billing_data['paperless_billing'].map({'Yes': True, 'No': False})  # Convert to boolean
insert_data('billing', billing_data)


C:\Users\Asus\AppData\Local\Temp\ipykernel_6484\381445300.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  billing_data['paperless_billing'] = billing_data['paperless_billing'].map({'Yes': True, 'No': False})  # Convert to boolean


![table.png](notebook_imgs/billing_filled.png)

### (iv) Inserting into Churn Predictions Table

In [45]:
# Prepare and insert data into Churn_Predictions table
prediction_data = prediction_results[['customerID', 'Churn']]
prediction_data.columns = ['customer_id', 'churn_prediction']

insert_data('churn_predictions', prediction_data)


![](notebook_imgs/churn_filled.png)

## 3. Querying the Data - Retrieving Customer Information Based on Various Criteria

In [6]:
import pandas as pd

### (i) Customers with tenure greater than x months

- There are 2058 customers with less than a tenure of one year
- There are 3105 customers with less than a tenure of two years
- There are 3833 customers with a tenure of more than two years
- Maximum tenure is of 72 months and there are 362 customers with this tenure
- Minimum tenure is of 1 months and there are 613 customers with this tenure


In [59]:
cur.execute("""
    SELECT * FROM Customers
    WHERE customer_id IN (
        SELECT customer_id FROM Billing
        WHERE tenure > 12
    );
            
""")

In [60]:
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description] #get column names from cursor

display_df = pd.DataFrame(rows, columns=colnames)

display_df


,customer_id,gender,senior_citizen,partner,dependents
0,5575-GNVDE,Male,False,False,False
1,7795-CFOCW,Male,False,False,False
2,1452-KIOVK,Male,False,False,True
3,7892-POOKP,Female,False,True,False
4,6388-TABGU,Male,False,False,True
...,...,...,...,...,...
4852,8456-QDAVC,Male,False,False,False
4853,2569-WGERO,Female,False,False,False
4854,6840-RESVB,Male,False,True,True
4855,2234-XADUH,Female,False,True,True


### (ii) Customers with monthly charges greater than x

- The lowest monthly charge is $18.25
- The maximum monthly charge is $118.75
- The average monthly charge is $64.7
- There are 3130 customers with monthly charges less than $65 (Average)
- There are 3897 customers with monthly charges greater than $65 (Average)


In [61]:
cur.execute("""
    SELECT * FROM Customers
    WHERE customer_id IN (
    SELECT customer_id FROM Billing
    WHERE monthly_charges > 50
);

""")

In [62]:
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description] #get column names from cursor

display_df = pd.DataFrame(rows, columns=colnames)

display_df

,customer_id,gender,senior_citizen,partner,dependents
0,5575-GNVDE,Male,False,False,False
1,3668-QPYBK,Male,False,False,False
2,9237-HQITU,Female,False,False,False
3,9305-CDSKC,Female,False,False,False
4,1452-KIOVK,Male,False,False,True
...,...,...,...,...,...
4734,7750-EYXWZ,Female,False,False,False
4735,6840-RESVB,Male,False,True,True
4736,2234-XADUH,Female,False,True,True
4737,8361-LTMKD,Male,True,True,False


### (iii) Customers with a specific contract type



- There are 3875 customers who opted for a Month-to-Month contract type
- There are 1472 customers who opted for a One Year contract type
- There are 1685 customers who opted for a Two Year contract type

In [8]:
cur.execute("""
    SELECT * FROM Customers
    WHERE customer_id IN (
    SELECT customer_id FROM Billing
    WHERE contract = 'Month-to-month'
);

""")

In [9]:
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description] #get column names from cursor

display_df = pd.DataFrame(rows, columns=colnames)

display_df

,customer_id,gender,senior_citizen,partner,dependents
0,7590-VHVEG,Female,False,True,False
1,3668-QPYBK,Male,False,False,False
2,9237-HQITU,Female,False,False,False
3,9305-CDSKC,Female,False,False,False
4,1452-KIOVK,Male,False,False,True
...,...,...,...,...,...
3870,9767-FFLEM,Male,False,False,False
3871,0639-TSIQW,Female,False,False,False
3872,8456-QDAVC,Male,False,False,False
3873,4801-JZAZL,Female,False,True,True


### (iv) Some more general information (Summary Reports)

In [19]:
cur.execute("""
    SELECT COUNT(*) AS total_customers FROM Customers;
""")

Total Number of Customers: 7032

In [22]:
cur.execute("""
    SELECT AVG(tenure) AS average_tenure FROM Billing;
""")

The Average Tenure is approx. 32 months

In [23]:
cur.execute("""
    SELECT SUM(monthly_charges * tenure) AS total_revenue FROM Billing;
""")

Total Revenue generated is $16,055,091.45

In [32]:
cur.execute("""
    SELECT COUNT(*) AS churned_customers
    FROM churn_predictions
    WHERE churn_prediction = TRUE;
""")

Number of predicted customers who have churned is 470

In [25]:
cur.execute("""
    SELECT b.contract, COUNT(cp.customer_id) AS churned_customers
    FROM Billing b
    JOIN Churn_Predictions cp ON b.customer_id = cp.customer_id
    WHERE cp.churn_prediction = TRUE
    GROUP BY b.contract;

""")

All of the predicted churned customers had a Month-to-Month contract type

In [26]:
cur.execute("""
    SELECT 
    AVG(B.monthly_charges) AS average_monthly_charges_churned
    FROM Churn_Predictions AS CP
    JOIN Billing AS B ON CP.customer_id = B.customer_id
    WHERE CP.churn_prediction = TRUE;

""")

Predicted customers who Churn have a average monthly charge of $81.37

In [ ]:
cur.execute("""
    SELECT 
    S.internet_service, COUNT(*) AS count
    FROM Churn_Predictions AS CP
    JOIN Services AS S ON CP.customer_id = S.customer_id
    WHERE CP.churn_prediction = TRUE
    GROUP BY S.internet_service;

""")

- 428 of the predicted churn customers had Fiber Optic
- 42 of the predicted churn customers had Fiber Optic

In [27]:
cur.execute("""
    SELECT 
    B.payment_method, COUNT(*) AS count
    FROM Churn_Predictions AS CP
    JOIN Billing AS B ON CP.customer_id = B.customer_id
    WHERE CP.churn_prediction = TRUE
    GROUP BY B.payment_method;

""")

- 344 of the predicted Churned customers used Electronic Checks as their payment method
- 47 of the predicted Churned customers used Mailed Checks as their payment method
- 34 of the predicted Churned customers used Credit Card as their payment method
- 45 of the predicted Churned customers used Bank transfer as their payment method

## 4. Storing Predictions

Lets assume we have a predictions dataframe (which we have gotten from running the pickle file on test dataset split and storing them into a df). 


NOTE: Since the customer_id field is referencing (foreign key) customer_id in Customers table, we need to ensure that the customerIDs here in the dataframe are also in that table!

In [44]:
predictions_df = pd.DataFrame({
    'customerID': ['7590-VHVEG', '5575-GNVDE', '3668-QPYBK'],
    'Churn': [True, False, True]
})

Now we can create a cell block or a function which stores the dataframe into our churn_predictions table

In [45]:
insert_query = """INSERT INTO Churn_Predictions (customer_id, churn_prediction) VALUES (%s, %s)"""

for i, row in predictions_df.iterrows():
    cur.execute(insert_query, (row['customerID'], row['Churn']))

![](notebook_imgs/insert_query.png)

## 5. Advanced SQL Analysis

Please look at the Summary Reports in the 3rd section of this notebook for detailed queries that drive insights related to customer churn

### Running Total 
A running total (also known as a cumulative sum) is the sum of a sequence of numbers that is updated each time a new number is added to the sequence. It gives the total at each point in the sequence.

In [47]:
cur.execute("""
    SELECT B.customer_id, B.monthly_charges,
    SUM(B.monthly_charges) OVER (ORDER BY B.customer_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total
    
    FROM Billing B;
       
""")

In [48]:
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description] #get column names from cursor

display_df = pd.DataFrame(rows, columns=colnames)

display_df


,customer_id,monthly_charges,running_total
0,0002-ORFBO,65.6,65.6
1,0003-MKNFE,59.9,125.5
2,0004-TLHLJ,73.9,199.4
3,0011-IGKFF,98.0,297.4
4,0013-EXCHZ,83.9,381.3
...,...,...,...
7027,9987-LUTYD,55.15,455398.75
7028,9992-RRAMN,85.1,455483.85
7029,9992-UJOEL,50.3,455534.15
7030,9993-LHIEB,67.85,455602.00


![](notebook_imgs/running_total.png)

### Moving Average
A moving average (also known as a rolling average) smooths out data by creating an average of different subsets of the full data set. It’s useful for identifying trends over time by filtering out short-term fluctuations.

In [49]:
cur.execute("""
    SELECT B.customer_id, B.monthly_charges,
    AVG(B.monthly_charges) OVER (ORDER BY B.customer_id ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS moving_average
    
    FROM Billing B;
 
""")

In [50]:
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description] #get column names from cursor

display_df = pd.DataFrame(rows, columns=colnames)

display_df


,customer_id,monthly_charges,moving_average
0,0002-ORFBO,65.6,65.6000000000000000
1,0003-MKNFE,59.9,62.7500000000000000
2,0004-TLHLJ,73.9,66.4666666666666667
3,0011-IGKFF,98.0,77.2666666666666667
4,0013-EXCHZ,83.9,85.2666666666666667
...,...,...,...
7027,9987-LUTYD,55.15,48.7500000000000000
7028,9992-RRAMN,85.1,53.7333333333333333
7029,9992-UJOEL,50.3,63.5166666666666667
7030,9993-LHIEB,67.85,67.7500000000000000


![](notebook_imgs/moving_averages.png)

## 6. Committing and Closing connection

In [40]:
cur.execute("ROLLBACK;")


In [46]:
conn.commit()

In [ ]:
cur.close()
conn.close()